In [1]:
import pandas as pd
import json
import numpy as np
from os import listdir
from os.path import isfile, join
import urllib
from collections import Counter
import re
from Bio import Phylo

# REDO

In [3]:
#### get list of files for HKY ####
path="/Users/maeperez/Desktop/HYPHY/redo/"
genes=[]
for file in listdir(path):
    if file[-4:]==".fna":
        genes+=[str(file.split('.')[0][8:])]
print len(genes),genes


7 ['Rmag_0146', 'Rmag_0045', 'Rmag_0043', 'Rmag_0514', 'Rmag_0502', 'Rmag_1053', 'Rmag_0566']


In [5]:
## KHtest ##
onlyfiles = [path+f for f in listdir(path) if f[-7:]=='.HKtest']

kh=pd.DataFrame([],columns=['Gene','p0.01','p0.05','p0.1'])
print 'number of genes that ran GARD',len(onlyfiles)
for i in range(len(onlyfiles)):
# for i in [0]:
#     print onlyfiles[i]
    with open(onlyfiles[i]) as f:
        gene=re.split('/|\.|aligned_',onlyfiles[i])[-3]
        lines=f.read().splitlines()
        for n in reversed(range(len(lines))):
            if lines[n][:6]=='At p =':
                try:
                    p001= float(lines[n-2].split(' ')[-3])
                    p005= float(lines[n-1].split(' ')[-3])
                    p01= float(lines[n].split(' ')[-3])
                except:
                    p001= np.nan
                    p005= np.nan
                    p01= np.nan
                break
        newentry=dict(zip(['Gene','p0.01','p0.05','p0.1'],[gene,p001,p005,p01]))
        kh=kh.append(newentry,ignore_index=True)

pd.set_option('display.max_rows', 500)
kh2=kh.copy()
for col in list(kh2)[1:]:
    kh2[col][kh2[col]>0] = float(col[1:])
    kh2[col][kh2[col]==0] = np.nan
kh2['KH_pval']= kh2.min(axis=1,numeric_only=True)


number of genes that ran GARD 8


/Users/maeperez/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maeperez/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
kh2

,Gene,p0.01,p0.05,p0.1,KH_pval
0,Rmag_0146,NaN,0.05,0.1,0.05
1,Rmag_0566,NaN,NaN,NaN,NaN
2,Rmag_0043,0.01,0.05,0.1,0.01
3,Rmag_1053,NaN,NaN,NaN,NaN
4,Rmag_0514,NaN,NaN,NaN,NaN
5,ls,NaN,NaN,NaN,NaN
6,Rmag_0045,NaN,NaN,NaN,NaN
7,Rmag_0502,NaN,NaN,NaN,NaN


In [6]:
df=pd.DataFrame([],columns=["R.magnifica", "Branch", 'Mean_dNdS','p_Holm','KH_pval'])
df

path="/Users/maeperez/Desktop/HYPHY/redo/"
# genes
# remote_file = sftp_client.open(path+'aligned_'+genes[i]+'.bsrel.json')
for i in range(len(genes)):
# for i in range(1):
    gene_id=genes[i]
    with open(path+'aligned_'+genes[i]+'.bsrel.json') as remote_file:
        bsrel_json = json.load (remote_file)
        for k in bsrel_json['test results'].keys():
#             new_row={'R.magnifica':gene_id,'Branch':k,'Mean_dNdS':bsrel_json['MG94']['rate distribution'][k],'p_Holm':bsrel_json['test results'][k]['p'],'KH_pval':kh2[kh2['Gene']==gene_id]['KH_pval'].get_values()[0]}
            new_row={'R.magnifica':gene_id,'Branch':k,'Mean_dNdS':bsrel_json['fits']['MG94']['rate distributions'][k].split(',')[0][2:],'p_Holm':bsrel_json['test results'][k]['p'],'KH_pval':kh2[kh2['Gene']==gene_id]['KH_pval'].get_values()[0]}

            df=df.append(new_row,ignore_index=True)
df['Branch']=[item.replace('_','.') for item in df['Branch']]

In [20]:
df[(df['p_Holm']<0.05)]


,R.magnifica,Branch,Mean_dNdS,p_Holm,KH_pval
12,Rmag_0146,Node4,0.0570578,0.00104391,0.05
65,Rmag_0043,Bathy.MS2017.0086.unknown.description.,0.0828508,0.0168819,0.01
66,Rmag_0043,SUP05.SP60.04245.unknown.description.,0.0794441,0.0106725,0.01
103,Rmag_0502,Node2,0.0251927,0.0167058,NaN
130,Rmag_1053,Node2,0.0854609,0.0224643,NaN


In [9]:
df[-1:]
df.to_csv('redo/absREL_results_redo.txt',sep='\t',header=True,index=False)        

In [8]:
#### color trees to show branches
from Bio import Phylo
import os
from os import listdir
from os.path import isfile, join

# mypath='/Users/maeperez/Desktop/HYPHY/GROUP2/'
mypath='/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/'

print len([val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk(mypath)] for val in sublist if val[-4:]=='.nwk'])


allfiles=[val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk(mypath)] for val in sublist if val[-4:]=='.nwk']
# print allfiles
with open('FigTree_Template.tree', 'r') as f:
    template=f.read()

# print template
for file in allfiles:
# for file in ['/Users/maeperez/Desktop/HYPHY/GROUP2/aligned_AAY24_RS01490.bsrel.annotated.nwk']
    Phylo.convert(file, 'newick', file[:-4]+'.nex', 'nexus')
# os.remove("ChangedFile.csv")

    with open(file[:-4]+'.nex', 'r') as f:
        tree=f.read()
        tree=tree.replace(':0.00000',':1.0')
#         print tree
        newtree=template.replace('!!!! YOUR TREE HERE !!!!',tree)
#         print newtree
        with open(file[:-4]+'.tree','w') as fa:
            fa.write(newtree)
    os.remove(file[:-4]+'.nex')
#     break

69


In [9]:
### build master nexus file ###
import re

mypath='/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/'

sample_names=['Bathy', 'SUP05', 'R_kilmeri', 'V_okutanii', 'V_diagonalis', 'V_extenta', 'V_gigas', 'R_pacifica', 'R_rectimargo', 'R_phaseoliformis', 'R_southwardae', 'R_magnifica', 'R_pleocardia']
    
print len([val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk(mypath)] for val in sublist if val[-4:]=='.nwk'])
print len([val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk(mypath)] for val in sublist if val[-15:]=='.annotated.tree'])

allfiles=sorted([val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk(mypath)] for val in sublist if val[-15:]=='.annotated.tree'])

print allfiles
treelist=''
i=0
for file in allfiles:
    gene=file.split('/')[-1].split('.')[0][8:]
    with open(file, 'r') as f:
        tree=f.read()
        start = 'Begin Trees;\n'
        end = 'End;\n'
        tree=tree[tree.find(start)+len(start):tree.rfind(end)]
        
        for sample in sample_names:
            tree=re.sub(sample+'.*?:',sample+':',tree, flags=re.DOTALL)
        
        tree=re.sub('Tree.*?=','Tree '+gene+'=',tree, flags=re.DOTALL)
        tree=re.sub('{MG[L0-9]}','',tree, flags=re.DOTALL)
        
    treelist+=tree
    i+=1
#     if i>10:
#         break

with open('FigTree_Template_MultiTree_Group1.tree', 'r') as f:
    template=f.read()
newtree=template.replace('!!!! YOUR TREE HERE !!!!',treelist)
with open('GROUP1_srvHKY.bsrel.annotated.tree','w') as fa:
    fa.write(newtree)


69
69
['/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0012.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0013.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0016.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0046.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0052.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0077.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0116.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0117.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0126.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0142.bsrel.annotated.tree', '/Users/maeperez/Desktop/HYPHY/server_Group1_HKY/aligned_Rmag_0146.bsrel.annotated.tree', '/U

In [17]:
path="/home/mp2c18/data/hyphy/absrel/"
for i in range(1):
# for i in range(1):
    gene_id=genes[i]
    print gene_id
    with sftp_client.open(path+'aligned_'+genes[i]+'.bsrel.json') as remote_file:
        bsrel_json = json.load (remote_file)
        for k in bsrel_json['test results'].keys():
            new_row={'R.magnifica':gene_id,'Branch':k,'Mean_dNdS':bsrel_json['fits']['MG94']['rate distributions'][k].split(',')[0][2:],'p_Holm':bsrel_json['test results'][k]['p'],'KH_pval':kh2[kh2['Gene']==gene_id]['KH_pval'].get_values()[0]}
            print new_row
#             df=df.append(new_row,ignore_index=True)
# df['Branch']=[item.replace('_','.') for item in df['Branch']]


Rmag_0012
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'0.1404892515609291', 'Branch': u'R_phaseoliformis', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'0.1078167575544285', 'Branch': u'SUP05', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'0.3082721593755137', 'Branch': u'R_southwardae', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'10', 'Branch': u'R_rectimargo', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'0.9469237018451497', 'Branch': u'Node21', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'10', 'Branch': u'V_extenta', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'0.464319564970941', 'Branch': u'R_magnifica', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'0.1666980225799917', 'Branch': u'V_gigas', 'p_Holm': 1}
{'R.magnifica': 'Rmag_0012', 'KH_pval': nan, 'Mean_dNdS': u'0.2172109426

# Mito

In [19]:
#### get list of files for HKY ####
path="/Users/maeperez/Desktop/HYPHY/mito/"
genes=[]
for file in listdir(path):
    if file[-4:]==".fna":
        genes+=[str(file.split('.')[0])]
print len(genes),genes


13 ['ATP6', 'CYTB', 'ND1', 'COX2', 'COX3', 'ND2', 'COX1', 'ND3', 'ND4L', 'ND6', 'ND4', 'ND5', 'ATP8']


In [20]:
## KHtest ##
onlyfiles = [path+f for f in listdir(path) if f[-7:]=='.HKtest']

kh=pd.DataFrame([],columns=['Gene','p0.01','p0.05','p0.1'])
print 'number of genes that ran GARD',len(onlyfiles)
for i in range(len(onlyfiles)):
# for i in [0]:
#     print onlyfiles[i]
    with open(onlyfiles[i]) as f:
        gene=re.split('/|\.|aligned_',onlyfiles[i])[-3]
        print(gene)
        lines=f.read().splitlines()
        for n in reversed(range(len(lines))):
            if lines[n][:6]=='At p =':
                try:
                    p001= float(lines[n-2].split(' ')[-3])
                    p005= float(lines[n-1].split(' ')[-3])
                    p01= float(lines[n].split(' ')[-3])
                except:
                    p001= np.nan
                    p005= np.nan
                    p01= np.nan
                break
        newentry=dict(zip(['Gene','p0.01','p0.05','p0.1'],[gene,p001,p005,p01]))
        kh=kh.append(newentry,ignore_index=True)

pd.set_option('display.max_rows', 500)
kh2=kh.copy()
for col in list(kh2)[1:]:
    kh2[col][kh2[col]>0] = float(col[1:])
    kh2[col][kh2[col]==0] = np.nan
kh2['KH_pval']= kh2.min(axis=1,numeric_only=True)


number of genes that ran GARD 13
ND3
ND6
CYTB
COX2
ND1
ND4
ATP6
ATP8
ND4L
ND2
COX1
ND5
COX3


/Users/maeperez/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maeperez/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
df=pd.DataFrame([],columns=["Gene", "Branch", 'Mean_dNdS','p_Holm','KH_pval'])
df

path="/Users/maeperez/Desktop/HYPHY/mito/"
# genes
# remote_file = sftp_client.open(path+'aligned_'+genes[i]+'.bsrel.json')
for i in range(len(genes)):
# for i in range(1):
    gene_id=genes[i]
    if gene_id=='ATP8':
        continue
    with open(path+genes[i]+'.bsrel.json') as remote_file:
        bsrel_json = json.load (remote_file)
        for k in bsrel_json['test results'].keys():
#             new_row={'R.magnifica':gene_id,'Branch':k,'Mean_dNdS':bsrel_json['MG94']['rate distribution'][k],'p_Holm':bsrel_json['test results'][k]['p'],'KH_pval':kh2[kh2['Gene']==gene_id]['KH_pval'].get_values()[0]}
            new_row={'Gene':gene_id,'Branch':k,'Mean_dNdS':bsrel_json['fits']['MG94']['rate distributions'][k].split(',')[0][2:],'p_Holm':bsrel_json['test results'][k]['p'],'KH_pval':kh2[kh2['Gene']==gene_id]['KH_pval'].get_values()[0]}

            df=df.append(new_row,ignore_index=True)
df['Branch']=[item.replace('_','.') for item in df['Branch']]

In [23]:
df[(df['p_Holm']<0.1)]


,Gene,Branch,Mean_dNdS,p_Holm,KH_pval
81,COX2,C.magnifica.Cmag.03.unknown.description.,0.170595,0.0113411,NaN
84,COX2,C.rectimargo.Crec.17.unknown.description.,0.452919,0.0116051,NaN
88,COX2,A.southwardae.Asou.18.unknown.description.,0.499089,0.0580853,NaN
90,COX2,C.pacifica.Cpac.17.unknown.description.,0.418019,0.00407456,NaN
94,COX2,Node2,0.135314,0.0650856,NaN
100,COX2,Node15,0.458578,1.89658e-09,NaN
105,COX2,Node24,0.435099,6.88245e-05,NaN
152,ND2,Node5,0.216422,0.0219767,NaN
315,ND5,Node6,0.0453839,0.0033657,NaN
